# Compare W2V models results over eval.set
## TODO: Weighted mean of vectors

In [13]:
# load eval.set (duplicates free)
import json
filename = 'pre-clustered_docs_harder.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)


# Load entities models
## TODO: Check entities format, does it really match the word in the vocabulary?

In [14]:
# load pre-trained model
from gensim.models import Word2Vec

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]

#wiki_model = Word2Vec.load('./models/wiki_iter=5_algorithm=skipgram_window=10_size=300_neg-samples=10.m')
my_model = Word2Vec.load('./models/w2v_entities+abstract_model.model')
print("Models loaded")

Models loaded


In [4]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
        elif word.lower() in index2word_set:
            nwords += 1.
            featureVec = np.add(featureVec, model[word.lower()])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(docs, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(docs),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for doc in docs:
       #
       # Print a status message every 1000th review
        #if counter%1000. == 0.:
        #    print("Review %d of %d" % (counter, len(docs)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(doc['result_entities'], model, \
           num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs
print(my_model['re'].shape)
trainDataVecs = getAvgFeatureVecs( unwrapped_docs, my_model, my_model.wv.vector_size )
print(trainDataVecs)
#urls_cluster_list = my_dbscan.perform_dbscan(eps = 0.4, min_samples = 4, data = trainDataVecs, verbose = True
#                  , titles = None, urls = None, print_noise = True)
db, clusters = my_dbscan.perform_dbscan(eps = 0.27, min_samples = 2, data = my_doc_vecs, verbose = True
                  , titles = titles, urls = urls, print_noise = True)
print(clusters, db)

(100,)
[[-0.4029415  -0.13100496  0.11882563 ... -0.33560228 -0.11999029
   0.24970573]
 [-0.33952245 -0.22362994  0.06498575 ... -0.28428683 -0.10469311
   0.16570613]
 [-0.3010722  -0.08241285  0.12940371 ... -0.35231194 -0.03992369
   0.22636156]
 ...
 [-0.45489705  0.24162287  0.19298626 ... -0.348121   -0.03367177
  -0.03407436]
 [-0.72975105  0.35789543  0.35934475 ... -0.50478923 -0.03901616
  -0.08851031]
 [-0.723315    0.32045954  0.33745176 ... -0.49936923 -0.03127839
  -0.07138227]]
{0: []} DBSCAN(algorithm='auto', eps=0.27, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=None, p=None)


/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [12]:
# check unknown words to model
model = my_model
unknown = []
n_words = 0
for doc in unwrapped_docs:
    for e in doc['result_entities']:
        n_words += 1
        try:
            model[e.lower()]
        except:
            try:
                model[e]
            except:
                unknown.append(e)
print(unknown[:30], len(unknown),'over', n_words)
#model['FUNGI']
model['sole']

['Crosta terrestre', 'Energia chimica', 'Università Brown', 'Bacteria', 'Fungi', 'Microrganismo', 'Processo chimico', 'Radioattività', 'Mars Odyssey', 'Impatto astronomico', 'Meteorite', 'Syrtis Major', 'Programma Mars', 'Stazione Spaziale Internazionale', 'Unione Sovietica', 'Atmosfera di Marte', 'Glucidi', 'Biomateriale', 'Plantae', 'Missioni con equipaggio su Marte', 'Guglielmo Braccio di Ferro', 'Colpo di genio', 'Anima gemella', 'Fiore delle Asteraceae', 'Romanticismo', "Dipartimento di Giustizia degli Stati Uniti d'America", 'Bloomberg News', 'Ipotesi di reato', 'Privatizzazione', 'Atto giuridico'] 321 over 1533


/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/home/nick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



array([ 0.0357947 ,  0.20191143,  0.78681916, -0.01710637,  0.06372207,
       -0.00410324, -0.14682736,  1.0912246 , -0.36416557,  0.16194542,
        0.08654545, -0.34655145,  0.6524546 , -0.1235489 ,  0.28006896,
        0.15622333,  0.03954355,  0.0188241 ,  0.04570109, -0.3862234 ,
        0.32317716, -0.37884653,  0.3571925 , -0.51524246,  0.1305867 ,
       -0.02990413, -0.4621041 , -0.39814064,  0.5153967 ,  0.86207205,
        0.82089263, -0.32117042, -0.6337599 , -0.15191874, -0.09647957,
       -0.41457617,  0.27958378,  0.14265476, -0.33186826,  0.01120468,
        0.13487722,  0.44845748,  0.87815243, -0.25615084, -0.12565109,
        0.04961443, -0.44201657,  0.17837808,  0.17717484, -0.07696109,
        0.32101938, -0.06838306, -0.23718631,  0.5409814 ,  0.00964856,
       -0.6696672 , -0.43719152,  0.29006076,  0.3100012 ,  0.30712837,
        0.0748999 , -0.10966552,  0.0045958 ,  0.0490436 ,  0.6589318 ,
        0.14339805,  0.4405871 , -0.2721798 , -0.12178715,  0.30

# Evaluate Clustering

In [16]:
import utils
import my_dbscan
import model_evaluation as me
import w2v_model_utils as mm_utils
import numpy as np
import importlib
importlib.reload(mm_utils)

# infer document vectors (once, they won't change)
# here's a problem we didn't consider: some docs might not have a representation at all 
# (all its entities are unknown to the model); we have to keep track of those.
my_doc_vecs = []
unknown_i = []
for i, doc in enumerate(unwrapped_docs):
    dvec = mm_utils.infer_vector(doc['result_entities'], my_model, verbose=True)
    if len(dvec) > 0:
        my_doc_vecs.append(dvec)
    else:
        print("Unknown doc found")
        unknown_i.append(i)
#print(my_doc_vecs[:2])
#wiki_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], wiki_model) for doc in unwrapped_docs]
# get titles and urls, ONLY from valid docs
titles = [doc['title'] for j, doc in enumerate(unwrapped_docs) if not(j in unknown_i)]
urls = [doc['url'] for j, doc in enumerate(unwrapped_docs) if not(j in unknown_i)]
assert len(titles) == len(urls)
print("Length of titles retrieved %s, and of docvecs %s" %(len(titles), len(my_doc_vecs)))

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples

# let's try 3 combinations of clustering
for k in range(3):
    # try different eps for some model
    if k == 0:
        # default eps dbscan
        eps = 0.27
        eps_increment = 0.1
    elif k==1:
        eps = 0.11
        eps_increment = 0.13
    else:
        # online update of model
        print("#######Online-training########")
        eps = 0.27
        eps_increment = 0.1
        training_corpus = [utils.lower_case_list(doc['result_entities']) for doc in unwrapped_docs]
        #training_corpus = [doc['result_entities'] for doc in unwrapped_docs]
        %time my_model = mm_utils.online_training(my_model, training_corpus)
        # wiki_model = mm_utils.online_training(wiki_model, training_corpus)
        # re-infer vectors
        my_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], my_model, verbose=True) for doc in unwrapped_docs]
        #wiki_doc_vecs = [mm_utils.infer_vector(doc['result_entities'], wiki_model) for doc in unwrapped_docs]
    doc_vecs = my_doc_vecs
    
    # TODO: change to 2 to cluster both models
    for i in range(1):
        # apply dbscan clustering to these vectors
        urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                                       urls = urls, subset_length = len(doc_vecs),
                                                     eps = eps, eps_increment = eps_increment,
                                                   n_iterations = 3, verbose = False, min_samples = min_s)
        # get clusters as list of titles
        titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
        clusters_found.append(titles_clusters)
        #data = utils.plot_clusters(titles_clusters)

        # evaluate clustering
        expected_clusters = []
        for docs in cdocs:
            expected_clusters.append([doc['title'] for doc in docs])
        precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
        print('Precision score: %s, Recall score: %s'%(precision, recall))

        models_precision.append(precision)
        models_recall.append(recall)
        # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
        # that's way we have precision score
        print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
            len(urls_cluster_list), len(cdocs)))
        # change inferred docs for next model
        #doc_vecs = wiki_doc_vecs

Known words: 56.666666666666664%  (17/30)
Known words: 77.77777777777777%  (7/9)
Known words: 82.6086956521739%  (19/23)
Known words: 88.23529411764706%  (15/17)
Known words: 100.0%  (4/4)
Known words: 85.71428571428571%  (12/14)
Known words: 84.61538461538461%  (11/13)
Known words: 66.66666666666667%  (10/15)
Known words: 77.77777777777777%  (14/18)
Known words: 80.76923076923077%  (21/26)
Known words: 53.57142857142857%  (15/28)
Known words: 52.27272727272727%  (23/44)
Known words: 100.0%  (7/7)
Known words: 90.47619047619048%  (19/21)
Known words: 75.38461538461539%  (49/65)
Known words: 95.0%  (19/20)
Known words: 85.24590163934427%  (52/61)
Known words: 92.3076923076923%  (12/13)
Known words: 88.88888888888889%  (16/18)
Known words: 91.48936170212765%  (86/94)
Known words: 93.33333333333333%  (14/15)
Known words: 87.5%  (14/16)
Known words: 65.38461538461539%  (17/26)
Known words: 90.9090909090909%  (10/11)
Known words: 94.44444444444444%  (17/18)
Known words: 51.724137931034484% 

In [11]:
for cluster in clusters_found[3]:
    print(cluster)
#print(unwrapped_docs[29])

{'fonte_dati': ['trend_analisys'], 'id': 'https://www.webnews.it/speciale/fifa-19/', 'ta_id': [6, 7], 'title': 'FIFA 19: novità, uscita e prezzo', 'abstract': 'Presentato ufficialmente in occasione della conferenza stampa all’E3 2018, FIFA 19 è la nuova iterazione del gioco di calcio sviluppato da EA Sports che costituisce il 27esimo titolo della popolare e vendutissima serie. Arriva sul mercato con tante novità, sia a livello di gameplay che delle competizioni disponibili, per sfidare ancora una volta PES 2019, il rivale di Konami.\n Novità e gameplay Modalità Carriera e campionati Il Viaggio FUT, ovvero FIFA Ultimate Team Comparto grafico Il testimonial in copertina: Cristiano Ronaldo La demo giocabile/a&gt; Prezzo e uscita \nFIFA 19 è stato nuovamente migliorato rispetto al precedente titolo della serie introducendo peraltro tante novità sia a livello di funzioni che di gameplay, che vanno a incrementare sostanzialmente le possibilità di gioco e la gestione in campo.\nNuovo sistema 

# Visualize results graphically

In [17]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3', api_key='FdMB4O6qCfciGDOnLvdQ')
graph_name = 'model_eval_entities_results'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since all the values are in between 0-1)
trace0 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_precision,
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x = model_descr + ['loaded model'],
    y = [s*100 for s in s_scores] + [0],
    name='Silhoutte score',
    marker=dict(
        color='rgb(204,50,100)',
    )
    
)

trace2 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_recall,
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    )
    
)
# precision, recall, silhoutte
data = [trace0, trace2,  trace1]
layout = go.Layout(
    title = 'Clustering Accuracy Results',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [16]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.27, eps_increment = 0.1, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.2057951
0.23791973
0.24932371
0.24040304
0.13758796
0.0041218703
0.12820731
0.228402


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.